# TP3 - Parte 3: Feature Engineering Avanzado

Ya habiendo realizado las visualizaciones, el Baseline, y acumulado experiencia trabajando con el dataset y las librerías de ML. Vamos a hacer un feature engineering más avanzado y robusto para luego utilizar esas features en modelos más avanzados.

Primero, aprendiendo de nuestro error, vamos a splitear previamente el dataset en train y validation.

Vamos a utilizar las columnas de keywords y location sin filtrar el target, y vamos a generar feature categóricas más interesantes. Que no sean binarias y que nos permitan aplicar algún encoding más avanzado de los que vimos en la materia. (Por ahora nuestras variables categóricas eran simplemente binarias y por eso podíamos aplicar un One-Hot encoding de forma directa sin tener que utilizar encoders específicos, ahora sí los usaremos).

A su vez, buscando un poco sobre el tema, encontre sobre [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), que se utiliza para aplicar varias transformaciones sobre distintos grupos de columnas de un DataFrame. Asique vamos a tratar de usarlo!

La idea final es aplicar las siguientes transformaciones:
- Hacer un target mean encoding de las `keywords`, como lo pide el enunciado.
- Identificar y normalizar de alguna forma las `locations`, la idea es ver si existee alguna herramienta útil para hacer esto, especializada para identificar ubicaciones geográficas. Luego, utilizar algún encoding apropiado (quizás binary encoding y/o frequency encoding)
- Mejorar las features numéricas y binarias previas. Un ejemplo es transformar `tweet_length` a una feature categórica de largos: "corto","medio","largo". Teniendo en cuenta que el largo máximo está limitado: El largo máximo de un tweet para la mayoría de los usuarios es de 280 caracteres. Sin embargo, los suscriptores de X Premium pueden publicar tuits más largos, con un límite de hasta 10.000 caracteres. Esto nos permitirá aplicar One-Hot Encoding (que lo pide el enunciado) en una feature que tiene sentido.
- Finalmente, reutilizar el embedding TF-IDF de `text`, que me pareció bastante útil y robusto. Quizás si me alcanza el tiempo, me gustaría agregar una búsqueda del hiperparámetro de cantidad de features. Esto dependerá de que tan friendly sea hacer esto utilizando `ColumnTransformer`.

Por último, voy a tener que elegir dos modelos avanzados.
- Creo que el TF-IDF que generé puede ser bastante bueno para calcular un KNN (dependiendo del `max_features`, ya que puedo tener algo con demasiadas dimensiones que dificulte computar las distancias). Además puedo sumar un One-Hot encoding de las categorías, me van a quedar muchisimas features pero luego puedo aplicar PCA para reducir las dimensiones y quedarme con las más útiles y que el modelo no sea tan costoso.
- También me gustaría probar un XGBoost, para ver cómo se comporta ya que me resultó un modelo interesante.
- Otra opción es probar con un Random Forest, aunque quizás no rinda muy bien. Para los modelos de arboles de decisión probablemente sea mejor usar un Target Mean Encoding y no tener tantas features binarias.

## Carga de datos y split en train y validation, preprocesado de test en paralelo.

Vamos a iniciar desde un primer momento con la carga de los datos y el split en train y validation, para no tener restricciones al generar features ni riesgo de filtrar nuestro target. A su vez, haremos el preprocesado de forma paralela en los tres sets, para no perder ningún paso intermedio.

In [ ]:
import pandas as pd
import numpy as np
import umap
import nltk

In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from sklearn.metrics import f1_score

In [ ]:
trainDf = pd.read_csv("../data/processed/TP3/train_modificado.csv", index_col=0)

In [ ]:
testDf = pd.read_csv("../data/processed/TP3/test_modificado.csv", index_col=0)

In [ ]:
trainDf.head(2)

In [ ]:
testDf.head(2)

Separamos en train, validation y también por X e y.

In [ ]:
X = trainDf.drop('target', axis=1)
y = trainDf['target']

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)

Como test no tiene target, ese dataframe directamente es X.

In [ ]:
X_test = testDf

## Feature categórico: Keywords
Las estandarizamos todas a minúscula y stripeamos. Y vemos cuantas hay. Si son muchas hacemos un Target Mean encoding o un binary, sino un one-hot

In [ ]:
X_train.keyword.nunique()

In [ ]:
X_train.keyword.map(lambda x: x.strip().lower() if type(x) == str else x).nunique()

Podemos ver que las keywords ya estan en lowercase y stripeadas (o aplicarles esto no junta distintas keywords), por lo tanto podemos usarlas como vienen.

In [ ]:
X_train[X_train.keyword.map(lambda x: '.' in x if type(x) == str else False)]

Además, corriendo la linea de arriba con varios caracteres podemos ver que no hay keywords compuestas (ej: rain,explosion). Por lo que un tweet no debe tener más de una única keyword.

Entonces, según el modelo que decidamos utilizar, tanto un One-Hot Encoding como un Target Mean Encoding tienen sentido. Siendo que solamente hay 221 features, un One-Hot es totalmente posible pero quizás para los árboles de decisión tener una única columna sea mejor (aunque esto puede perder información).

## Feature categórico: Location

In [ ]:
trainDf['location'].nunique()

En este caso, tenemos 3341 ubicaciones distintas, lo que se torna demasiadas categorías como para hacer un One-Hot.

Vamos a trabajar con X_train para no filtrar el target.

In [ ]:
locations = X_train[['location']].value_counts()

In [ ]:
locations.info()

Tenemos 2554 ubicaciones distintas. Veamos las mas comunes:

In [ ]:
locations.nlargest(30)

In [ ]:
for i in range (1,10):
  print(f"Cantidad de ubicaciones con al menos {i+1} apariciones: ", len(locations[locations > i]))

Podemos ver que solo 446 ubicaciones se repiten al menos 1 vez. Por lo que las que no se repiten no nos servirían. Pero quizás podemos utilizar alguna herramienta especializada en detectar ubicaciones. Con eso podemos juntar ubicaciones similares y luego sí aplicar un encoding.

Con esto en mente y buscando en internet + Chats IA, una solución recomendada es utilizar Hybrid NER + Geodecoding.

**Hybrid NER**
Un Named Entity Recognition (NER), como su nombre indica, es un método que se utiliza para reconocer entidades en texto. Se llaman hibridos a aquellos NER que combinan métodos basados en reglas con métodos basados en aprendizaje automático para realizar el reconocimiento.

**Geodecoding**
Geodecoding es el proceso de extraer ubicaciones mencionadas en texto y convertirlas en coordenadas geográficas.

Genial! Lo que tenía en mente era realizar simplemente algo del estilo NER, pero la recomendación de utilizar Geodecoding nos permitirá además hacer que las ubicaciones no sean simplemente categorías, sion que también la distancia entre ellas tenga sentido. Veremos qué tan díficil es aplicar esto, en caso de que represente una complejidad demasiado alta, me voy a quedar simplemente con el hybrid NER.

Resulta que el problema que tenemos es que queremos unificar sinónimos. Para eso, NER nos permite linkear sinónimos que representan la misma ubicación geográfica a esa entidad.

Según ChatGPT:
> spaCy puede detectar entidades tipo GPE / LOC incluso cuando hay palabras adicionales.
> Ej.:
> - “new york, ny” → “New York”
> - “washington, dc area” → “Washington”

Vamos a ver cómo se comporta `spaCy`. A su vez, como vimos algunas repeticiones fáciles entre las locations más comunes, vamos a usar un mapeo manual para estos casos triviales. Si luego de aplicar spacy, el resultado no puede mapearse manualmente, entonces vamos a pasarlo por un Geodecoding (usando Nominatim) que es una herramienta para obtener la versión estandarizada de la ubicación.

Por otro lado, un conflicto que surgió usando estas herramientas fue decidir la precisión y el formato en el que nos guardamos la ubicación detectada. Para esta ocación como quiero formar una variable categórica, decidí quedarme con la ubicación más precisa y solo con esa. Por lo tanto, nos quedamos con la ciudad antes que el estado y con el estado antes que el país. Y siempre nos quedamos con uno solo de ellos.

In [ ]:
import spacy
from geopy.geocoders import Nominatim

In [ ]:
nlp = spacy.load("en_core_web_sm")
geolocator = Nominatim(user_agent="geo")

In [ ]:
def extract_location_old(text):
  if type(text) != str:
    return None
  doc = nlp(text)
  for ent in doc.ents:
      if ent.label_ in ("GPE", "LOC"):
          return ent.text
  return None

In [ ]:
def canonicalize(loc):
    if loc is None:
        return None
    try:
        result = geolocator.geocode(loc, addressdetails=True)
        if result and "address" in result.raw:
            addr = result.raw["address"]
            # Regla: si es ciudad → devolver "City, Country"
            if "city" in addr:
                return addr["city"]
            # Si no hay ciudad pero sí estado
            if "state" in addr:
                return addr["state"]
            # sino, me quedo con el país
            if "country" in addr:
                return addr["country"]
    except:
        pass
    return loc

In [ ]:
MANUAL_MAP = {
    "usa": "United States of America",
    "us": "United States of America",
    "united states": "United States of America",
    "uk": "United Kingdom",
    "united kingdom": "United Kingdom",
    "nyc": "New York",
    "new york": "New York",
    "new york, ny": "New York",
    "los angeles": "Los Angeles",
    "los angeles, ca": "Los Angeles",
    "london": "London",
}

Para ver como funciona, vamos a testear manualmente con `thirty_largest_locations`:

In [ ]:
thirty_largest_locations = locations.nlargest(30).reset_index().location
thirty_largest_locations

In [ ]:
thirty_largest_locations.map(extract_location_old)

Acá podemos sacar una conclusión interesante: spaCy no aporta gran valor para la feature `location` en los casos más comunes, ya que estos datos ya están bastante normalizados y por lo tanto pueden pasarse directamente al mapeo manual + Nominatim.

De igual forma, vamos a utilizarlo ya que para las locations menos comunes, con una sola aparición, puede llegar a extraer alguna ubicación. Pero vamos a mejorar la función `extract_location` para que en caso de no encontrar ubicación con spaCy, haga un fallback y devuelva la ubicación original.



In [ ]:
def extract_location(text: str):
  doc = nlp(text)
  for ent in doc.ents:
      if ent.label_ in ("GPE", "LOC"):
          return ent.text
  return text

In [ ]:
def normalize_location(text):
    if type(text) != str:
        return None

    t = text.lower().strip()

    # 1. Diccionario manual
    if t in MANUAL_MAP:
        return MANUAL_MAP[t]

    # 2. NER
    ner_loc = extract_location(t)
    if ner_loc and ner_loc.lower() in MANUAL_MAP:
        return MANUAL_MAP[ner_loc.lower()]

    # 3. Geodecoding
    return canonicalize(ner_loc or t)

In [ ]:
thirty_largest_locations.map(normalize_location)

Con esto, ya parece que tenemos un transformador de sinónimos bastante potable que podemos aplicar a nuestro set de datos! Vamos a utilizarlo:

In [ ]:
X_train['standard_location'] = X_train.location.map(normalize_location)

In [ ]:
X_train.standard_location.nunique()

In [ ]:
X_train.location.nunique()

Excelente! (Después de +1h de procesado 😰) Pasamos de unas 2500 ubicaciones a tan solo 1444. Disminuimos en más de 1000 la cantidad de categorias. Vamos a aplicar la misma transformación a X_validation y X_test, ya que estas transformaciones dependen unicamente de la row en sí misma y no guardan ninguna relación con el target.

In [ ]:
X_validation['standard_location'] = X_validation.location.map(normalize_location)

In [ ]:
X_validation.standard_location.nunique()

In [ ]:
X_validation.location.nunique()

In [ ]:
X_test['standard_location'] = X_test.location.map(normalize_location)

In [ ]:
X_test.standard_location.nunique()

In [ ]:
X_test.location.nunique()

Con esto, ya hicimos un preprocesado muy bueno de la feature location, de tipo categórica a la que podremos aplicarle embeddings distintos para los distintos modelos.

Un detalle que faltó observar, es si había presencia o no de ubicaciones de string vacío. Por lo que revisaremos y en caso de que haya quedado alguna, vamos a decidir considerarlas como ubicaciones nulas, ya que esto puede engañar al modelo a pensar que las ubicaciones vacías en realidad son una ubicación válida

In [ ]:
len(X_train[X_train.standard_location == ''])

In [ ]:
X_train['standard_location'] = X_train['standard_location'].replace('', np.nan)

In [ ]:
len(X_train[X_train.standard_location == ''])

In [ ]:
X_validation['standard_location'] = X_validation['standard_location'].replace('', np.nan)

In [ ]:
X_test['standard_location'] = X_test['standard_location'].replace('', np.nan)

OBSERVACIÓN: Un uso interesante para spaCy podría ser crear un nuevo feature "location_from_text" y usar spaCy para extraer ubicaciones del texto de los tweets, ya que puede haber algunos que digan la ubicación pero no la tengan formateada a la columna correspondiente. Sin embargo no vamos a realizarlo por ahora.

OBSERVACIÓN 2: Otro uso extra de Nominatim, sería el de utilizarlo para obtener la longitud y latitud de cada una de las categorías. Lo que podría considerarse un Geospatial Embedding, que nos permitirá obtener distancias euclideas entre las palabras con sentido. Voy a dejar esto como un approach opcional si tengo tiempo. Esto probablemente sería muy bueno para un Random Forest.

## Feature de Texto: Text

Para el campo text, vamos a aplicar un embedding TF-IDF como venimos haciendo. Con la diferencia de que ajustaremos la cantidad de max_features según sea mejor para el modelo, y trataremos también de que sea un hiperparámetro a buscar. Vamos a seguir usando TweetTokenizer.

## Refinando las Features Anteriores

Repasemos las features numéricas que teníamos:

In [ ]:
X_train.head(1)

Ya tenemos las features categóricas de keyword y location. Vamos a agregar un feature categórico extra: tweet_length. Vamos a separarlo en 3 categorias: corto, medio y largo.

El embedding que hicimos con el TweetTokenizer nos sirve para medir los hashtags y urls más comunes, asique vamos a pasar a tener un count de la cantidad de carácteres interesantes: #, @, urls (numéricos en vez de categóricos).

Y además vamos a agregar dos features más que nos hablaran de la estructura del tweet:

- proporción de espacios vs caracteres totales.
- proporción de palabras vs caracteres totales.

In [ ]:
X_train["tweet_length"].describe()

Tenemos los percentiles de tweet_lentgh. Asique podemos transformarla en una categórica de 4 largos posibles para mayor comodidad: short, medium_short, medium_long y long.

Para no filtrar info de test ni de validation. Vamos a usar los mismos percentiles de X_train para todos.

In [ ]:
Q1 = 78
Q2 = 107
Q3 = 133

In [ ]:
def bucket_length(x):
    if x <= Q1:
        return "short"
    elif x <= Q2:
        return "medium_short"
    elif x <= Q3:
        return "medium_long"
    else:
        return "long"

In [ ]:
X_train["tweet_length"] = X_train["tweet_length"].map(bucket_length)

In [ ]:
X_validation["tweet_length"] = X_validation["tweet_length"].map(bucket_length)

In [ ]:
X_test["tweet_length"] = X_test["tweet_length"].map(bucket_length)

Ahora dropeamos las features binarias:

In [ ]:
cols_to_drop = ["has_hashtag", "has_url", "has_tag"]

X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

In [ ]:
cols_to_drop = ["has_hashtag", "has_url", "has_tag"]
X_test = X_test.drop(columns=cols_to_drop)

Generamos las features numericas respectivas:

In [ ]:
X_train["num_hashtags"] = X_train["text"].str.count(r"#\w+")
X_validation["num_hashtags"] = X_validation["text"].str.count(r"#\w+")
X_test["num_hashtags"] = X_test["text"].str.count(r"#\w+")

In [ ]:
X_train["num_urls"] = X_train["text"].str.count(r"(http[s]?://\S+|www\.\S+)")
X_validation["num_urls"] = X_validation["text"].str.count(r"(http[s]?://\S+|www\.\S+)")
X_test["num_urls"] = X_test["text"].str.count(r"(http[s]?://\S+|www\.\S+)")

In [ ]:
X_train["num_tags"] = X_train["text"].str.count(r"@\w+")
X_validation["num_tags"] = X_validation["text"].str.count(r"@\w+")
X_test["num_tags"] = X_test["text"].str.count(r"@\w+")

In [ ]:
X_test["num_hashtags"] = X_test["text"].str.count(r"#\w+")
X_test["num_urls"] = X_test["text"].str.count(r"(http[s]?://\S+|www\.\S+)")
X_test["num_tags"] = X_test["text"].str.count(r"@\w+")

Y ahora las proporciones:

In [ ]:
X_train['total_chars'] = X_train['text'].str.len()

In [ ]:
X_validation['total_chars'] = X_validation['text'].str.len()

In [ ]:
X_test['total_chars'] = X_test['text'].str.len()

In [ ]:
X_train['prop_digits']  = X_train['num_digits'] / X_train['total_chars']
X_validation['prop_digits']  = X_validation['num_digits'] / X_validation['total_chars']
X_test['prop_digits']  = X_test['num_digits'] / X_test['total_chars']

In [ ]:
X_train['prop_words'] = X_train['words_count'] / X_train['total_chars']
X_validation['prop_words'] = X_validation['words_count'] / X_validation['total_chars']
X_test['prop_words'] = X_test['words_count'] / X_test['total_chars']

In [ ]:
X_test['prop_digits']  = X_test['num_digits'] / X_test['total_chars']
X_test['prop_words'] = X_test['words_count'] / X_test['total_chars']

Finalmente tenemos:

In [ ]:
X_train.info()

Dropeamos las features que no vamos a utilizar:
location ya la estandarizamos, total_chars la tenemos por tweet_length categórica y words_count es menos información que prop_words asique decido descartar esas para no tener features redundantes.

In [ ]:
X_train = X_train.drop(columns=["location","total_chars","words_count"])
X_validation = X_validation.drop(columns=["location","total_chars","words_count"])
X_test = X_test.drop(columns=["location","total_chars","words_count"])

In [ ]:
X_test = X_test.drop(columns=["location","total_chars","words_count"])

Y para finalizar, como el estandarizado de locations consumió mucho tiempo. Voy a persistir los datos para evitar perdida de los mismos y facilitar la ejecución de los modelos. De paso, chequeo que sigan los indices en el orden correspondiente:

## Persistencia de Datos

In [ ]:
X_train.index[:4]

In [ ]:
y_train.index[:4]

In [ ]:
X_train.to_csv("../data/processed/TP3/X_train_procesado.csv", index=True)
y_train.to_csv("../data/processed/TP3/y_train_procesado.csv", index=True)

In [ ]:
print(X_validation.index[:4])
print(y_validation.index[:4])

In [ ]:
X_validation.to_csv("../data/processed/TP3/X_validation_procesado.csv", index=True)
y_validation.to_csv("../data/processed/TP3/y_validation_procesado.csv", index=True)

En test no tengo los y.

In [ ]:
X_test.to_csv("../data/processed/TP3/X_test_procesado.csv", index=True)

Ahora, con el feature engineering final ya realizado. Ya podemos utilizar ColumnTransformer para los embeddings/encodings y mezclarlo con Pipeline y Cross-Validation para entrenar los modelos elegidos.

# TP3 - Catalogando los features

In [ ]:
X_train.info()

In [ ]:
cols_numericas = [
  "num_uppercase_letters",
  "num_uppercase_words",
  "num_special_chars",
  "num_digits",
  "num_hashtags",
  "num_urls",
  "num_tags",
  "prop_digits",
  "prop_words"
]

In [ ]:
cols_categoricas = [
    "keyword",
    "tweet_length",
    "standard_location"
]

In [ ]:
cols_textuales = [
    "text"
]

# TP3 - Feature Engineering Avanzado 2

## Cambio de Embedding: BERTweet

Realizando los modelos de ML, surgieron preprocesados nuevos o distintos que tuve que probar para mejorar los modelos.

Primero, el embedding TF-IDF resultó poco útil para los modelos (sobretodo para KNN y XGBoost que fueron los primeros modelos que probé) incluso usando el TweetTokenizer. Luego, utilicé word2vec para el embedding de KNN, pero tampoco dio muchos resultados.

Por lo tanto busqué otro embedding que fuese mejor para este problema y encontré [BERTweet](https://huggingface.co/docs/transformers/model_doc/bertweet).

> BERTweet shares the same architecture as BERT-base, but it’s pretrained like RoBERTa on English Tweets. It performs really well on Tweet-related tasks like part-of-speech tagging, named entity recognition, and text classification.

A su vez, la doc de BERT-base dice:

> BERT is a bidirectional transformer pretrained on unlabeled text to predict masked tokens in a sentence and to predict whether one sentence follows another. The main idea is that by randomly masking some tokens, the model can train on text to the left and right, giving it a more thorough understanding. BERT is also very versatile because its learned language representations can be adapted for other NLP tasks by fine-tuning an additional layer or head.

Por lo tanto, decidí utilizar BERTweet como embedding para el campo text, ya que genera únicamente 768 dimensiones y es un mucho mejor embedding que TF-IDF.

Entonces, para el modelo de XGBoost hice este nuevo embedding y ya noté una mejoría porque empezó dando scores de 0.70-0.71 en validation. Los embeddings los corrí en ese colab pero traigo para acá las celdas para mayor prolijidad y orden.

Después, persistí los embeddings en archivos .csv y en las celdas de abajo se puede ver cómo utilizar estos archivos para anexar este embedding a nuestro set de datos, luego en ColumnTransformer utilizamos `remainder="passthrough"` para que las features de BERTweet pasen directamente al modelo.


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", model_max_length=512)
model = AutoModel.from_pretrained("vinai/bertweet-base")
model = model.to(device)

def embed_texts(texts):
    all_embeddings = []
    batch_size = 16  # evitar out-of-memory

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tokens = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**tokens)

        cls_batch = outputs.last_hidden_state[:,0,:].numpy()
        all_embeddings.append(cls_batch)

    return np.vstack(all_embeddings)

X_train_bert = embed_texts(X_train["text"].tolist())
X_validation_bert = embed_texts(X_validation["text"].tolist())
X_test_bert = embed_texts(X_test["text"].tolist())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
bert_cols = [f"bert_{i}" for i in range(768)]

df_train_bert = pd.DataFrame(X_train_bert, columns=bert_cols, index=X_train.index)
df_valid_bert = pd.DataFrame(X_validation_bert, columns=bert_cols, index=X_validation.index)
df_test_bert = pd.DataFrame(X_test_bert, columns=bert_cols, index=X_test.index)

## Persistencia de BERTweet Embedding

Este embedding es bastante costoso de computar, por lo tanto los exportamos a archivos CSV para evitar hacerlo constantemente

In [ ]:
df_train_bert.to_csv("../data/processed/TP3/df_train_bert.csv", index=True)
df_valid_bert.to_csv("../data/processed/TP3/df_valid_bert.csv", index=True)

In [ ]:
df_test_bert.to_csv("../data/processed/TP3/df_test_bert.csv", index=True)

Y ahora cada vez que necesitemos usarlo lo cargamos de los archivos, lo concatenamos al set X y dropeamos la columna de texto porque ya se le realizó el embedding:

In [ ]:
df_train_bert = pd.read_csv("../data/processed/TP3/df_train_bert.csv", index_col=0)
df_valid_bert = pd.read_csv("../data/processed/TP3/df_valid_bert.csv", index_col=0)
df_test_bert = pd.read_csv("../data/processed/TP3/df_test_bert.csv", index_col=0)

In [ ]:
X_train = pd.concat([X_train, df_train_bert], axis=1)
X_validation = pd.concat([X_validation, df_valid_bert], axis=1)
X_test = pd.concat([X_test, df_test_bert], axis=1)

In [ ]:
X_train = X_train.drop(columns=["text"])
X_validation = X_validation.drop(columns=["text"])
X_test = X_test.drop(columns=["text"])

## Geodecoding: Latitud y Longitud

Después de trabajar con los modelos Baseline, KNN y XGBoost, creo que mis features requieren más información, por lo tanto decidí extraer la latitud y longitud de las ubicaciones que ya tengo estandarizadas. Para eso vamos a volver a utilizar Nominatim.

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="geo")

In [ ]:
def get_lat_lon(location):
  try:
    loc = geolocator.geocode(location)
    if loc:
      return pd.Series([loc.latitude, loc.longitude])
    else:
      return pd.Series([None, None])
  except:
    return pd.Series([None, None])

In [ ]:
X_train_latlon = X_train["standard_location"].apply(get_lat_lon)
X_train_latlon.columns = ["lat","lon"]
X_train_latlon.index = X_train.index

In [ ]:
X_train_latlon.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6090 entries, 4392 to 10090
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     5355 non-null   float64
 1   lon     5355 non-null   float64
dtypes: float64(2)
memory usage: 142.7 KB


In [ ]:
X_train_latlon.shape

(6090, 2)

In [ ]:
X_train_latlon[X_train_latlon['lat'] != None].head(2)

,lat,lon
id,,
4392,NaN,NaN
59,NaN,NaN


In [ ]:
X_validation_latlon = X_validation["standard_location"].apply(get_lat_lon)
X_validation_latlon.columns = ["lat","lon"]
X_validation_latlon.index = X_validation.index

In [ ]:
X_test_latlon = X_test["standard_location"].apply(get_lat_lon)
X_test_latlon.columns = ["lat","lon"]
X_test_latlon.index = X_test.index

Explorando los resultados, me dí cuenta que Nominatim le asignó las coordenadas (latitud: 34.220389, longitud: 70.380031) a los valores nulos de "standard_location" porque procesó el NaN como "nan" y asignó esas coordenadas por algún motivo. Cómo este procesado consumió 3hs y no tengo ese tiempo para volver a ejecutarlo, tomo la decisión de reemplazar todas esas coordenadas en los dataframes de coordenadas por valores nulos.

In [ ]:
X_train_latlon.loc[[7135,1454,6399,5248]]

,lat,lon
id,,
7135,34.220389,70.380031
1454,34.220389,70.380031
6399,34.220389,70.380031
5248,34.220389,70.380031


In [ ]:
X_train.loc[[7135,1454,6399,5248]]

,keyword,text,tweet_length,num_uppercase_letters,num_uppercase_words,num_special_chars,num_digits,standard_location,num_hashtags,num_urls,num_tags,prop_digits,prop_words
id,,,,,,,,,,,,,
7135,military,@UniversityofLaw For the people who died in Hu...,long,16,0,11,7,NaN,0,2,1,0.05000,0.121429
1454,body%20bagging,@amaramin3 Meek is definitely capable of body ...,medium_long,2,0,2,1,NaN,0,0,1,0.00885,0.185841
6399,hurricane,Them shootas be so hungry with bodies on they ...,short,1,0,2,0,NaN,0,0,0,0.00000,0.200000
5248,fatality,Fatality https://t.co/GF5qjGoyCi,short,5,0,5,1,NaN,0,1,0,0.03125,0.062500


Si veo la cantidad de filas con esa latitud en latlot:

In [ ]:
(X_train_latlon.lat == 34.220389).sum()

np.int64(2070)

Y cuento la cantidad de valores nulos en la respectiva columna satandard_location

In [ ]:
X_train.standard_location.info()

<class 'pandas.core.series.Series'>
Index: 6090 entries, 4392 to 10090
Series name: standard_location
Non-Null Count  Dtype 
--------------  ----- 
4020 non-null   object
dtypes: object(1)
memory usage: 224.2+ KB


In [ ]:
6090 - 4020

2070

Vemos que el resultado es el mismo. Asique puedo simplemente reemplazar esas filas por NaN's.

In [ ]:
X_train_latlon.lat = X_train_latlon.lat.replace(34.220389, np.nan)
X_train_latlon.lon = X_train_latlon.lon.replace(70.3800314, np.nan)

In [ ]:
X_validation_latlon.lat = X_validation_latlon.lat.replace(34.220389, np.nan)
X_validation_latlon.lon = X_validation_latlon.lon.replace(70.3800314, np.nan)

In [ ]:
X_test_latlon.lat = X_test_latlon.lat.replace(34.220389, np.nan)
X_test_latlon.lon = X_test_latlon.lon.replace(70.3800314, np.nan)

Y finalmente, persistimos los datos de latitudes y longitudes:

In [ ]:
X_validation_latlon.to_csv("../data/processed/TP3/X_validation_latlon.csv", index=True)
X_train_latlon.to_csv("../data/processed/TP3/X_train_latlon.csv", index=True)
X_test_latlon.to_csv("../data/processed/TP3/X_test_latlon.csv", index=True)

In [ ]:
X_test_latlon.to_csv("../data/processed/TP3/X_test_latlon.csv", index=True)

Además, esto me hizo dar cuenta que por algún motivo que desconozco, la columna de "standard_location" de test.csv está completamente nula:

In [ ]:
X_test.standard_location.info()

<class 'pandas.core.series.Series'>
Index: 3263 entries, 0 to 10875
Series name: standard_location
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 51.0 KB


Investigué los valores originales de location de X_test y encontré que:

In [ ]:
testDf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3263 entries, 0 to 10875
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   keyword                3237 non-null   object
 1   location               2142 non-null   object
 2   text                   3263 non-null   object
 3   tweet_length           3263 non-null   int64 
 4   words_count            3263 non-null   int64 
 5   num_uppercase_letters  3263 non-null   int64 
 6   num_uppercase_words    3263 non-null   int64 
 7   num_special_chars      3263 non-null   int64 
 8   num_digits             3263 non-null   int64 
 9   has_hashtag            3263 non-null   int64 
 10  has_url                3263 non-null   int64 
 11  has_tag                3263 non-null   int64 
dtypes: int64(9), object(3)
memory usage: 331.4+ KB


Hay entradas no nulas y que claramente deberían tener un valor asignado. Por lo tanto voy a volver a calcular la standard_location para test. Debe haber ocurrido un error en la persistencia de los datos.

Asique volví a ejecutar todas las transformaciones y celdas de X_test para volver a aplicarlas y encontré el bug (estaba pisando X_test con X_train cuando hacía el replace de strings vacías ''). Y ahora el resultado quedó:

In [ ]:
X_test.standard_location.info()

<class 'pandas.core.series.Series'>
Index: 3263 entries, 0 to 10875
Series name: standard_location
Non-Null Count  Dtype 
--------------  ----- 
2139 non-null   object
dtypes: object(1)
memory usage: 51.0+ KB


También persistí X_test volviendo a ejecutar la celda de guardado del mismo.

Otro fallo que encontré es que en algunos indices hay ubicaciones que deberían tener un standard_location valido y no lo tienen. Por ejemplo en testDf, la entrada con id=46 es longon

In [ ]:
testDf[testDf.location == 'london'].head(1)

,keyword,location,text,tweet_length,words_count,num_uppercase_letters,num_uppercase_words,num_special_chars,num_digits,has_hashtag,has_url,has_tag
id,,,,,,,,,,,,
46,ablaze,london,Birmingham Wholesale Market is ablaze BBC News...,120,16,18,1,7,0,0,1,0


In [ ]:
X_test[X_test.standard_location.str.contains("ondon", na=False)].head(5)

,keyword,text,tweet_length,num_uppercase_letters,num_uppercase_words,num_special_chars,num_digits,standard_location,num_hashtags,num_urls,num_tags,prop_digits,prop_words
id,,,,,,,,,,,,,
46,ablaze,Birmingham Wholesale Market is ablaze BBC News...,medium_long,18,1,7,0,London,0,1,0,0.000000,0.133333
188,aftershock,Brass and Copper in Cataclysm &amp; AfterShock...,short,8,0,10,0,London,0,1,0,0.000000,0.111111
1046,bleeding,@TomBrevoort 'Bleeding Cool as read by Tom Bre...,medium_short,7,0,5,0,London,0,0,1,0.000000,0.170213
1642,bombing,What it was like to survive the atomic bombing...,medium_short,6,0,5,3,London,0,1,0,0.036585,0.146341
1674,bridge%20collapse,BREAKING NEWS: Australia collapse to a hapless...,medium_short,21,2,6,3,London,0,1,0,0.031250,0.145833


Pero tiene coordenadas nulas en X_test_latlon. Y hay otras entradas que tienen "london" con X_test_latlon válido. Esto no se me ocurrió antes pero puede ser que se deba a los TIMEOUTs que nos dió Nominatim, yo pensé que hacía retrys pero se ve que no los hace.

In [ ]:
X_test_latlon[X_test_latlon.lat.notna()].head()

,lat,lon
id,,
51,9.600036,7.999972
60,34.053691,-118.242766
70,38.895037,-77.036543
75,22.351115,78.667743
87,43.653482,-79.383935


Una mejora para el TP sería buscar parámetros para prevenir estos timeouts, en ese caso se obtendrían mejores ubicaciones y latitudes y longitudes para los tweets. Que podrían llevarnos a mejores resultados. Pero no me alcanzó el tiempo para volver a ejecutar estas celdas porque consumen varias horas. Una mejora muy importante sería hacer una optimicación a la función de get_lat_lon, para que solo consulte en Nominatim una vez por cada standard_location única.

# TP3 - Modelo 1: KNN (Descartado por score. Se reutilizó para la parte 4)

Celda para carga de datasets:

In [ ]:
X_train = pd.read_csv("../data/processed/TP3/X_train_procesado.csv", index_col=0)
y_train = pd.read_csv("../data/processed/TP3/y_train_procesado.csv", index_col=0)

X_validation = pd.read_csv("../data/processed/TP3/X_validation_procesado.csv", index_col=0)
y_validation = pd.read_csv("../data/processed/TP3/y_validation_procesado.csv", index_col=0)

X_test = pd.read_csv("../data/processed/TP3/X_test_procesado.csv", index_col=0)

Para nuestro modelo KNN vamos a comenzar sin utilizar la feature de location. Simplemente vamos a usar:
- el One-Hot encoding de Keyword
- el TF-IDF embedding de text
- las features numéricas refinadas

A su vez, para simplificar el código y que sea más prolijo y profesional vamos a utilizar Pipeline y ColumnTransformer lo que nos permitirá "encolar" los pasos de preprocesado, búsqueda de hiperparámetros y training.

Además, el One-Hot nos va a dar 221 features y el TF-IDF muchas más, lo que nos va a dejar una matriz muy dispersa y con muchas dimensiones, algo que empeora mucho el performance de KNN, asique vamos a usar una reducción de dimensiones t-SNE o UMAP para esto.

In [ ]:
scaler = StandardScaler()

In [ ]:
tweet_tok = TweetTokenizer()

def tweet_tokenizer(text):
    return tweet_tok.tokenize(text)

In [ ]:
stopwordsEng = list(stopwords.words('english'))

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['keyword']),
        ('tfidf', TfidfVectorizer(
            lowercase = True,
            tokenizer=tweet_tokenizer,
            token_pattern = None,
            stop_words = stopwordsEng,
            max_features=2000       # valor inicial, será hiperparámetro
        ),
        'text'),
        ('num', StandardScaler(), cols_numericas)
    ],
    remainder='drop'
)

In [ ]:
umap_reducer = umap.UMAP(n_components=500, random_state=42, n_jobs=1)

In [ ]:
pipeline = Pipeline([
    ('preproc', preprocessor),
    ('reducer', umap_reducer),
    ('knn', KNeighborsClassifier())
])

Para optimizar los tiempos y tener resultados más estables, pensé en dividir la búsqueda de hiperparámetros en dos etapas:
1. Para buscar los parámetros del Vectorizer y el UMAP, ya que son independientes de KNN y pueden introducir muchisima variabilidad en los resultados de KNN y empeorar el performance.
2. Luego, optimizar los hiperparámetros propios de KNN.

A su vez, voy a estimar el tiempo de realizar un fit, para saber cuántas combinaciones puedo llegar a manejar y no ejecutar celdas que puedan demorar demasiadas horas. De paso testeamos el predict contra validation para ver cómo le va:

In [ ]:
import time
start = time.time()
pipeline.fit(X_train, y_train)  # usa un subset pequeño
end = time.time()
print("Tiempo estimado fit:", end-start)

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Tiempo estimado fit: 170.55051612854004


In [ ]:
y_pred = pipeline.predict(X_validation)

In [ ]:
pipeline.named_steps['knn'].get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
f1 = f1_score(y_validation, y_pred)
print("F1-score en validation de lr default:", f1)

F1-score en validation de lr default: 0.6156274664561957


Buscamos opciones de hiperparámetros para la primera parte.

## Ajustando TF-IDF y UMAP

### Iteración 1

In [ ]:
params_tfidf_umap = {
    'preproc__tfidf__max_features': [1000, 1500, 2000, 2500, 3000],
    'reducer__n_components': [100, 200, 400],
}

In [ ]:
grid_tfidf_umap = GridSearchCV(
    pipeline,
    param_grid=params_tfidf_umap,
    cv=2,
    verbose=2
)

In [ ]:
grid_tfidf_umap.fit(X_train, y_train)

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] END preproc__tfidf__max_features=1500, reducer__n_components=100; total time=  31.1s
[CV] END preproc__tfidf__max_features=1500, reducer__n_components=100; total time=  21.7s
[CV] END preproc__tfidf__max_features=1500, reducer__n_components=200; total time=  26.6s
[CV] END preproc__tfidf__max_features=1500, reducer__n_components=200; total time=  34.1s
[CV] END preproc__tfidf__max_features=1500, reducer__n_components=400; total time=  50.5s
[CV] END preproc__tfidf__max_features=1500, reducer__n_components=400; total time=  55.0s
[CV] END preproc__tfidf__max_features=2000, reducer__n_components=100; total time=  18.5s
[CV] END preproc__tfidf__max_features=2000, reducer__n_components=100; total time=  18.6s
[CV] END preproc__tfidf__max_features=2000, reducer__n_components=200; total time=  28.9s
[CV] END preproc__tfidf__max_features=2000, reducer__n_components=200; total time=  29.4s
[CV] END preproc__tfidf__max_features=2

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['keyword']),
                                                                        ('tfidf',
                                                                         TfidfVectorizer(max_features=2000,
                                                                                         stop_words=['a',
                                                                                                     'about',
                                                                                                     'above',
                                                                                                     'after',
                                                                                                     'again',
                                                                                                     'against',
                                                                                                     'ain',
                                                                                                     'all',
                                                                                                     'am',
                                                                                                     'an',
                                                                                                     'and',
                                                                                                     'any',
                                                                                                     'are',
                                                                                                     'aren',
                                                                                                     "aren't",
                                                                                                     'as',
                                                                                                     'at',
                                                                                                     'be',
                                                                                                     'because',
                                                                                                     'been',
                                                                                                     'before',
                                                                                                     'being...
                                                                          'prop_words'])])),
                                       ('reducer',
                                        UMAP(n_components=500, n_jobs=1, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'preproc__tfidf__max_features': [1500, 2000, 2500,
                                                          3000],
                         'reducer__n_components': [100, 200, 400]},
             verbose=2)

In [ ]:
y_pred = grid_tfidf_umap.predict(X_validation)

f1 = f1_score(y_validation, y_pred)
print("F1-score en validation de lr default:", f1)

F1-score en validation de lr default: 0.6131850675138999


El mejor resultado lo obtuvimos con n_components=400 y max_features=1500. Con un score menor que antes, asique vamos a volver a entrenar con otros parámetros.

### Iteración 2

In [ ]:
params_tfidf_umap = {
    'preproc__tfidf__max_features': [1250, 1500, 1750, 2000],
    'reducer__n_components': [200, 300, 400, 500, 600],
}

In [ ]:
grid_tfidf_umap = GridSearchCV(
    pipeline,
    param_grid=params_tfidf_umap,
    cv=2,
    verbose=2
)

In [ ]:
grid_tfidf_umap.fit(X_train, y_train)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=200; total time=  41.3s
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=200; total time=  27.2s
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=300; total time=  39.2s
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=300; total time=  36.8s
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=400; total time=  49.2s
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=400; total time=  49.5s
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=500; total time= 1.1min
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=500; total time= 1.1min
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=600; total time= 1.4min
[CV] END preproc__tfidf__max_features=1250, reducer__n_components=600; total time= 1.4min
[CV] END preproc__tfidf__max_features=1

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['keyword']),
                                                                        ('tfidf',
                                                                         TfidfVectorizer(max_features=2000,
                                                                                         stop_words=['a',
                                                                                                     'about',
                                                                                                     'above',
                                                                                                     'after',
                                                                                                     'again',
                                                                                                     'against',
                                                                                                     'ain',
                                                                                                     'all',
                                                                                                     'am',
                                                                                                     'an',
                                                                                                     'and',
                                                                                                     'any',
                                                                                                     'are',
                                                                                                     'aren',
                                                                                                     "aren't",
                                                                                                     'as',
                                                                                                     'at',
                                                                                                     'be',
                                                                                                     'because',
                                                                                                     'been',
                                                                                                     'before',
                                                                                                     'being...
                                                                          'prop_words'])])),
                                       ('reducer',
                                        UMAP(n_components=500, n_jobs=1, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'preproc__tfidf__max_features': [1250, 1500, 1750,
                                                          2000],
                         'reducer__n_components': [200, 300, 400, 500, 600]},
             verbose=2)

In [ ]:
y_pred = grid_tfidf_umap.predict(X_validation)

f1 = f1_score(y_validation, y_pred)
print("F1-score en validation de lr default:", f1)

F1-score en validation de lr default: 0.6053882725832013


El mejor resultado lo obtuvimos con n_components=300 y max_features=1250. Con un score menor que antes. Este ajuste no está sirviendo. Es posible que la reducción de dimensiones con UMAP esté empeorando mucho el KNN. Vamos a probar sin usar UMAP pero disminuyendo los features del embedding.

### Iteración 3

In [ ]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['keyword']),
        ('tfidf', TfidfVectorizer(
            lowercase = True,
            tokenizer=tweet_tokenizer,
            token_pattern = None,
            stop_words = stopwordsEng,
            max_features=4000
        ),
        'text'),
        ('num', StandardScaler(), cols_numericas)
    ],
    remainder='drop'
)

In [ ]:
pipeline2 = Pipeline([
    ('preproc', preprocessor2),
    ('knn', KNeighborsClassifier())
])

Testeo con un solo fit para ver cuánto tardará:

In [ ]:
start = time.time()
pipeline2.fit(X_train, y_train)
y_pred = pipeline2.predict(X_validation)
f1 = f1_score(y_validation, y_pred)
print("F1-score en validation de KNN default:", f1)
end = time.time()
print("Tiempo estimado fit + predict:", end-start)

F1-score en validation de KNN default: 0.6225806451612903
Tiempo estimado fit + predict: 2.4309356212615967


Reviso los params default:



In [ ]:
pipeline2.named_steps['knn'].get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

Y ahora vamos a hacer el grid search. El fit base nos demoró casi 3mins. Tenemos 16 combinaciones de hiperparámetros lo que nos da unos 50mins por cada CV. Asique vamos a hacer 3 CV y dejar ejecutando un rato largo. Voy a usar joblib para guardar el estimador optimo obtenido:

In [ ]:
import joblib

In [ ]:
params = {
    'knn__n_neighbors': [3,5,7,9,11,13,15],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1,2]
}

In [ ]:
grid_2 = GridSearchCV(
    pipeline2,
    param_grid=params,
    cv=3,
    verbose=2
)

In [ ]:
grid_2.fit(X_train, y_train)

Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] END .knn__n_neighbors=3, knn__p=1, knn__weights=uniform; total time=   2.2s
[CV] END .knn__n_neighbors=3, knn__p=1, knn__weights=uniform; total time=   3.1s
[CV] END .knn__n_neighbors=3, knn__p=1, knn__weights=uniform; total time=   2.3s
[CV] END knn__n_neighbors=3, knn__p=1, knn__weights=distance; total time=   2.0s
[CV] END knn__n_neighbors=3, knn__p=1, knn__weights=distance; total time=   2.0s
[CV] END knn__n_neighbors=3, knn__p=1, knn__weights=distance; total time=   2.0s
[CV] END .knn__n_neighbors=3, knn__p=2, knn__weights=uniform; total time=   1.3s
[CV] END .knn__n_neighbors=3, knn__p=2, knn__weights=uniform; total time=   1.4s
[CV] END .knn__n_neighbors=3, knn__p=2, knn__weights=uniform; total time=   2.0s
[CV] END knn__n_neighbors=3, knn__p=2, knn__weights=distance; total time=   1.9s
[CV] END knn__n_neighbors=3, knn__p=2, knn__weights=distance; total time=   1.3s
[CV] END knn__n_neighbors=3, knn__p=2, knn__weig

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['keyword']),
                                                                        ('tfidf',
                                                                         TfidfVectorizer(max_features=4000,
                                                                                         stop_words=['a',
                                                                                                     'about',
                                                                                                     'above',
                                                                                                     'after',
                                                                                                     'again',
                                                                                                     'against',
                                                                                                     'ain',
                                                                                                     'all',
                                                                                                     'am',
                                                                                                     'an',
                                                                                                     'and',
                                                                                                     'any',
                                                                                                     'are',
                                                                                                     'aren',
                                                                                                     "aren't",
                                                                                                     'as',
                                                                                                     'at',
                                                                                                     'be',
                                                                                                     'because',
                                                                                                     'been',
                                                                                                     'before',
                                                                                                     'being...
                                                                                         tokenizer=<function tweet_tokenizer at 0x7b1180c65f80>),
                                                                         'text'),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['num_uppercase_letters',
                                                                          'num_uppercase_words',
                                                                          'num_special_chars',
                                                                          'num_digits',
                                                                          'num_hashtags',
                                                                          'num_urls',
                                                                          'num_tags',
                        

In [ ]:
y_pred = grid_2.predict(X_validation)

f1 = f1_score(y_validation, y_pred)
print("F1-score en validation de grid_2:", f1)

F1-score en validation de grid_2: 0.6129917657822507


Para guardarlo:

In [ ]:
joblib.dump(grid_2.best_estimator_, "mejor_modelo.pkl")

Y para cargarlo:

In [ ]:
modelo = joblib.load("mejor_modelo.pkl")

El mejor resultado lo obtuvimos con n_components=? y max_features=?. Con un score menor que antes, asique vamos a volver a entrenar con otros parámetros.

### Cambiando el embedding:

El TF-IDF claramente no está dando buenos resultados. Voy a probar con word2vec a ver si me da mejores resultados.

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 34.9 MB/s eta 0:00:00


In [ ]:
from gensim.models import Word2Vec

Entreno word2vec

In [ ]:
tweet_tok = TweetTokenizer()

def tokenize(t):
    return tweet_tok.tokenize(t.lower())

sentences = X_train["text"].apply(tokenize).tolist()

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)

creo el FunctionTransformer

In [ ]:
def w2v_embedding(text_series):
    def embed_sentence(tokens):
        vecs = [w2v_model.wv[w] for w in tokens if w in w2v_model.wv]
        if len(vecs) == 0:
            return np.zeros(w2v_model.vector_size)
        return np.mean(vecs, axis=0)

    tokenized = text_series.apply(tokenize)
    embedded = np.vstack(tokenized.apply(embed_sentence).values)
    return embedded

embed_transformer = FunctionTransformer(w2v_embedding, validate=False)

Nuevo ColumnTransformer:

In [ ]:
preprocessor3 = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['keyword']),
        ('w2v', embed_transformer, 'text'),
        ('num', StandardScaler(), cols_numericas)
    ],
    remainder='drop'
)

In [ ]:
pipeline3 = Pipeline([
    ('preproc', preprocessor3),
    ('knn', KNeighborsClassifier())
])

In [ ]:
params = {
    'knn__n_neighbors': [3,5,7,9,11,13,15],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1,2]
}

In [ ]:
grid_3 = GridSearchCV(
    pipeline3,
    param_grid=params,
    cv=3,
    verbose=2
)

In [ ]:
y_train.head(1)

In [ ]:
grid_3.fit(X_train, y_train)

Fitting 3 folds for each of 28 candidates, totalling 84 fits


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV] END .knn__n_neighbors=3, knn__p=1, knn__weights=uniform; total time=   4.8s


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV] END .knn__n_neighbors=3, knn__p=1, knn__weights=uniform; total time=   5.3s


...

[CV] END knn__n_neighbors=15, knn__p=2, knn__weights=distance; total time=   1.2s


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV] END knn__n_neighbors=15, knn__p=2, knn__weights=distance; total time=   1.1s


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['keyword']),
                                                                        ('w2v',
                                                                         FunctionTransformer(func=<function w2v_embedding at 0x79ab1bbc6ac0>),
                                                                         'text'),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['num_uppercase_letters',
                                                                          'num_uppercase_words',
                                                                          'num_special_chars',
                                                                          'num_digits',
                                                                          'num_hashtags',
                                                                          'num_urls',
                                                                          'num_tags',
                                                                          'prop_digits',
                                                                          'prop_words'])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
                         'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']},
             verbose=2)

In [ ]:
y_pred = grid_3.predict(X_validation)

f1 = f1_score(y_validation, y_pred)
print("F1-score en validation de grid_3:", f1)

F1-score en validation de grid_3: 0.5949367088607594


## Conclusiones

En este punto, decidí descartar esta versión de KNN. Entrenarlo consumía mucho tiempo, los features no eran buenos (TF-IDF es un embedding de demasiadas dimensiones para KNN, los features numéricos no aportaban mucho y el OneHot encoding de keyword ayudaba mucho menos. Además no se utilizó la location para nada)

Entonces pasé al modelo XGBoost que se puede ver en el colab [XGBoost](https://colab.research.google.com/drive/1ni2_w-Useb1sDnNnvfFXUPBi5euFh-Jw?usp=sharing). En él usé un embedding de texto mucho mejor (basado en BERT) y me involucré más con el set de datos, y con eso obtuve un modelo con un puntaje mucho mejor y más cercano al objetivo de 0.8.

Luego de que XGBoost se acercara pero no lograra llegar al puntaje deseado y obtuviera mucho overfitting, llegué a la conclusión de que debía obtener mejores features. Asique me puse en un nuevo [colab](https://colab.research.google.com/drive/1eG9ansDIJDkN28k-wDn9jcKrvFtBsqh7?usp=sharing) a implementar un modelo Random Forest. Esta vez agregué a las features la extracción de la latitud y longitud en base a las ubicaciones estandarizadas y agregué features de relación basadas en la categórica keyword que mejoraron muchisimo la perfomance.